In [48]:
import pandas as pd
from pandasql import sqldf
import random


In [49]:
extract = pd.read_csv("final_output.csv")
extract


,team,role
0,1,9
1,1,6
2,1,2
3,1,5
4,1,8
...,...,...
485,98,9
486,98,3
487,98,10
488,98,6


In [50]:
df = sqldf("""
SELECT
role1,
role2,
role3,
role4,
role5,
role6,
role7,
role8,
role9,
role10,
role11
FROM
    (SELECT
      team,
      SUM(CASE WHEN role = 1 THEN 1 ELSE 0 END) AS role1,
      SUM(CASE WHEN role = 2 THEN 1 ELSE 0 END) AS role2,
      SUM(CASE WHEN role = 3 THEN 1 ELSE 0 END) AS role3,
      SUM(CASE WHEN role = 4 THEN 1 ELSE 0 END) AS role4,
      SUM(CASE WHEN role = 5 THEN 1 ELSE 0 END) AS role5,
      SUM(CASE WHEN role = 6 THEN 1 ELSE 0 END) AS role6,
      SUM(CASE WHEN role = 7 THEN 1 ELSE 0 END) AS role7,
      SUM(CASE WHEN role = 8 THEN 1 ELSE 0 END) AS role8,
      SUM(CASE WHEN role = 9 THEN 1 ELSE 0 END) AS role9,
      SUM(CASE WHEN role = 10 THEN 1 ELSE 0 END) AS role10,
      SUM(CASE WHEN role = 11 THEN 1 ELSE 0 END) AS role11
    FROM extract
    GROUP BY 1
    ) a
""", locals())
df['target'] = "_0"
df



,role1,role2,role3,role4,role5,role6,role7,role8,role9,role10,role11,target
0,0,1,0,0,1,1,0,1,1,0,0,_0
1,1,1,0,0,0,2,0,0,0,1,0,_0
2,1,0,1,0,0,1,0,0,1,0,1,_0
3,0,1,0,0,1,1,0,1,0,0,1,_0
4,0,1,0,1,1,0,0,1,1,0,0,_0
...,...,...,...,...,...,...,...,...,...,...,...,...
93,1,1,0,0,1,1,0,0,1,0,0,_0
94,0,1,1,0,0,2,1,0,0,0,0,_0
95,1,0,0,0,1,1,0,0,1,0,1,_0
96,1,1,0,0,0,0,0,1,1,0,1,_0


In [51]:
# I realize stratified sampling could work but doing this
for index, row in df.iterrows():
    roles = []
    for i in range(1, 12):
        if row[f'role{i}'] == 1:
            roles.append(i)
    random_choice = random.choice(roles)
    df.at[index, 'target'] = "_" + chr(random_choice+97)
    df.at[index, f'role{random_choice}'] = 0
    

In [52]:
def add_three_numbers(a, b, c):
    return a + b + c

df['tank_or_catcher'] = df[['role1', 'role7', 'role8']].apply(lambda x: add_three_numbers(*x), axis=1).astype(float)
# df.drop(['role1','role7','role8'], axis='columns', inplace=True)
df['diver_juggernaut_or_melee'] = df[['role2', 'role3', 'role9']].apply(lambda x: add_three_numbers(*x), axis=1).astype(float)
df.drop(['role2','role3','role9'], axis='columns', inplace=True)


In [53]:
df

,role1,role4,role5,role6,role7,role8,role10,role11,target,tank_or_catcher,diver_juggernaut_or_melee
0,0,0,1,0,0,1,0,0,_g,1.0,2.0
1,1,0,0,2,0,0,1,0,_c,1.0,0.0
2,1,0,0,0,0,0,0,1,_g,1.0,2.0
3,0,0,1,1,0,1,0,1,_c,1.0,0.0
4,0,1,1,0,0,0,0,0,_i,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...
93,1,0,1,1,0,0,0,0,_j,1.0,1.0
94,0,0,0,2,0,0,0,0,_h,0.0,2.0
95,0,0,1,1,0,0,0,1,_b,0.0,1.0
96,1,0,0,0,0,1,0,1,_c,2.0,1.0


In [54]:
from numpy.random import seed
from sklearn.model_selection import train_test_split
seed(32)
split_size = 0.4 if df.shape[0] < 100000 else 40000/df.shape[0]
train, temp = train_test_split(df, test_size=split_size)
dev, test = train_test_split(temp, test_size=0.50)
X_train = train.drop(['target'], axis=1)
y_train = train[['target']]
X_dev = dev.drop(['target'], axis=1)
y_dev = dev[['target']]
X_test = test.drop(['target'], axis=1)
y_test = test[['target']]
X = df.drop(['target'], axis=1)
y = df[['target']]
train.shape, dev.shape, test.shape, train.target.value_counts(normalize=True)


((58, 11),
 (20, 11),
 (20, 11),
 _g    0.189655
 _c    0.172414
 _b    0.155172
 _d    0.103448
 _j    0.086207
 _i    0.068966
 _h    0.068966
 _f    0.051724
 _l    0.051724
 _e    0.051724
 Name: target, dtype: float64)

In [55]:
from sklearn.metrics import accuracy_score

def get_score():
    seed(32)
    model.fit(X_train, y_train)
    accuracy_train = accuracy_score(y_train, model.predict(X_train))
    accuracy_dev = accuracy_score(y_dev, model.predict(X_dev))
    print("Train Score:", accuracy_train)
    print("Dev Score:", accuracy_dev)
accuracy_baseline1 = accuracy_score(y_dev, [5]*X_dev.shape[0])
accuracy_baseline2 = accuracy_score(y_dev, [6]*X_dev.shape[0])
accuracy_baseline3 = accuracy_score(y_dev, [8]*X_dev.shape[0])
print("Baseline Score1:", accuracy_baseline1)
print("Baseline Score2:", accuracy_baseline2)
print("Baseline Score3:", accuracy_baseline3)

Baseline Score1: 0.0
Baseline Score2: 0.0
Baseline Score3: 0.0


In [56]:
dev

,role1,role4,role5,role6,role7,role8,role10,role11,target,tank_or_catcher,diver_juggernaut_or_melee
17,1,0,1,0,0,0,0,1,_g,1.0,1.0
26,1,0,1,0,1,0,0,0,_g,2.0,1.0
49,1,0,1,1,0,0,0,0,_j,1.0,1.0
22,0,1,0,0,1,0,0,0,_l,1.0,2.0
13,1,0,0,2,0,0,0,0,_f,1.0,1.0
83,1,0,0,0,0,1,0,1,_k,2.0,1.0
94,0,0,0,2,0,0,0,0,_h,0.0,2.0
15,1,0,0,2,0,0,0,0,_j,1.0,1.0
61,1,0,1,0,0,0,0,0,_g,1.0,2.0
75,2,0,0,1,0,1,0,0,_c,3.0,0.0


In [57]:
from sklearn import linear_model, neighbors, tree, svm, ensemble
from xgboost import XGBClassifier
# Algorithms
algorithms = []
# algorithms.append(linear_model.LogisticRegression())
# algorithms.append(linear_model.SGDClassifier())
# algorithms.append(neighbors.KNeighborsClassifier(n_neighbors=1))
# algorithms.append(neighbors.KNeighborsClassifier(n_neighbors=3))
# algorithms.append(neighbors.KNeighborsClassifier(n_neighbors=5))
# algorithms.append(neighbors.KNeighborsClassifier(n_neighbors=7))
# algorithms.append(neighbors.KNeighborsClassifier(n_neighbors=11))
algorithms.append(tree.DecisionTreeClassifier(max_depth=3))
# algorithms.append(svm.SVC(kernel='linear'))
# algorithms.append(svm.SVC(kernel='poly'))
# algorithms.append(svm.SVC(kernel='rbf'))
# algorithms.append(svm.SVC(kernel='sigmoid'))
# algorithms.append(XGBClassifier(learning_rate=0.1, max_depth=10, min_child_weight=13, n_estimators=40, nthread=1, subsample=0.55))


In [58]:
for algorithm in algorithms:
    algorithm.fit(X_train, y_train)
    pred = algorithm.predict(X_dev)
    print(accuracy_score(pred, y_dev))
    

0.2


In [59]:
model = tree.DecisionTreeClassifier(max_depth=3)
model.fit(X_train, y_train)
pred = algorithm.predict(X_dev)
print(accuracy_score(pred, y_dev))

0.2


In [60]:
model = tree.DecisionTreeClassifier(max_depth=3)

In [61]:
model.fit(X, y)

DecisionTreeClassifier(max_depth=3)

In [62]:
import graphviz
import os

dot_data = tree.export_graphviz(model, out_file=None,
                      feature_names=train.drop(["target"], axis=1).columns,
                      class_names=['vanguard', 'diver', 'juggernaut', 'enchanter', 'adc', 'mage', 'warden', 'catcher', 'melee', 'battlemage', 'assassin'],
                      filled=True, rounded=True,
                      special_characters=True)
graph = graphviz.Source(dot_data)
graph.render()

os.remove("Source.gv")
os.rename("Source.gv.pdf", "decision_tree.pdf")


In [63]:
sorted(y.values), len(set(y))

([array(['_b'], dtype=object),
  array(['_b'], dtype=object),
  array(['_b'], dtype=object),
  array(['_b'], dtype=object),
  array(['_b'], dtype=object),
  array(['_b'], dtype=object),
  array(['_b'], dtype=object),
  array(['_b'], dtype=object),
  array(['_b'], dtype=object),
  array(['_b'], dtype=object),
  array(['_b'], dtype=object),
  array(['_b'], dtype=object),
  array(['_c'], dtype=object),
  array(['_c'], dtype=object),
  array(['_c'], dtype=object),
  array(['_c'], dtype=object),
  array(['_c'], dtype=object),
  array(['_c'], dtype=object),
  array(['_c'], dtype=object),
  array(['_c'], dtype=object),
  array(['_c'], dtype=object),
  array(['_c'], dtype=object),
  array(['_c'], dtype=object),
  array(['_c'], dtype=object),
  array(['_c'], dtype=object),
  array(['_d'], dtype=object),
  array(['_d'], dtype=object),
  array(['_d'], dtype=object),
  array(['_d'], dtype=object),
  array(['_d'], dtype=object),
  array(['_d'], dtype=object),
  array(['_d'], dtype=object),
  array(

In [64]:
y.value_counts()

target
_g        19
_c        13
_b        12
_j        10
_h         9
_f         9
_l         8
_d         8
_i         5
_e         3
_k         2
dtype: int64